In [3]:
pip install requests_html

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install lxml_html_clean

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyppeteer==1.0.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import random 
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
# import nest_asyncio
import pyppeteer
import time
from urllib.parse import urljoin
import urllib
import pandas as pd


import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()

USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0 Safari/537.36',
]

headers = {
    'User-Agent': random.choice(USER_AGENTS),
}
asession = AsyncHTMLSession()

In [2]:
product_links = []
min_pages = 1
max_pages = 3

for x in range (min_pages, max_pages+1): 
    url = "https://www.lazada.sg/shop-mobiles/?clickTrackInfo=f7fd6c1a-e30e-439a-9ecf-2bc335bb14f5__3__2374658766__20__0.1__333258__7253__c2i__0.0__38%2525%2Boff&from=hp_categories&item_id=2374658766&params=%7B%22catIdLv1%22%3A%222%22%2C%22pvid%22%3A%22f7fd6c1a-e30e-439a-9ecf-2bc335bb14f5%22%2C%22src%22%3A%22ald%22%2C%22categoryName%22%3A%22Mobiles%22%2C%22categoryId%22%3A%223%22%7D&q=Mobiles&spm=a2o42.homepage.categoriesPC.d_1_3&src=hp_categories&up_id=2374658766&version=v2&page="
    response = await asession.get(url+ str(x))
    await response.html.arender(sleep=2, timeout=20, reload=True)
    soup = BeautifulSoup(response.html.html, 'html.parser')
    
    product_list = soup.find_all('div', class_ = "Bm3ON")

    for product in product_list: 
        
        link = product.find("a")['href']
        link = link.replace("//", "https://")
        product_links.append(link)

data = []
for link in product_links: 
    f = requests.get(link, headers=headers).text
    results = BeautifulSoup(f, 'html.parser')
    
    try: 
        name = results.find("h1", {"class" : "pdp-mod-product-badge-title"})
        if name is not None:
            name = name.get_text("\\n").strip()
    except: 
        name = None

    try: 
        price = results.find("span", {"class" : "notranslate pdp-price pdp-price_type_normal pdp-price_color_orange pdp-price_size_xl"}).text
    except: 
        price = None
    
    try: 
        num_ratings = results.find("a", {"class" : "pdp-link pdp-link_size_s pdp-link_theme_blue pdp-review-summary__link"}, first=True)
        if num_ratings is not None: 
            num_ratings = name.get_text("\\n").strip()
    except: 
        num_ratings = None
    
    try: 
        stock_count = results.find("span", {"class" : "quantity-content-default"})
        if stock_count is not None: 
            stock_count = stock_count.get_text("\\n").strip()
    except: 
        stock_count = None
    
    # try:
    #     seller = results.find("a", {"class" : "pdp-link pdp-link_size_l pdp-link_theme_black seller-name__detail-name"})
    #     if seller is not None: 
    #         seller = seller.get_text("\\n").strip()
    # except: 
    #     seller = None
    
    # try: 
    #     seller_ratings = results.find("div", {"class" : "seller-info-value rating-positive"})
    #     if seller_ratings is not None: 
    #         seller_ratings = seller_ratings.get_text("\\n").strip()
    # except: 
    #     seller_ratings = None
    
    product_info = {
        "name" : name,
        "price" : price,
        "number of reviews" : num_ratings,
        "available_stock" : stock_count,


    }
    data.append(product_info)
    

[D:urllib3.connectionpool] Starting new HTTPS connection (%d): %s:%s
[D:urllib3.connectionpool] %s://%s:%s "%s %s %s" %s %s
[D:websockets.client] = connection is CONNECTING
[D:websockets.client] > GET %s HTTP/1.1
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] > %s: %s
[D:websockets.client] < HTTP/1.1 %d %s
[D:websockets.client] < %s: %s
[D:websockets.client] < %s: %s
[D:websockets.client] < %s: %s
[D:websockets.client] < %s: %s
[D:websockets.client] = connection is OPEN
[D:websockets.client] > %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] > %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] > %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets.client] < %s
[D:websockets

In [ ]:
df = pd.DataFrame(data, columns = ['Name', 'Price', "Number of Reviews", "Available Stock", "Seller", "Seller Ratings"])

In [ ]:
df.to_csv("LazadaProducts.csv")